## Lab Assignment One: Exploring Table Data

#### Luis Garduno

### 1. Business Understanding

##### About League of Legends :
Developed by Riot Studios, League of Legends, or "LoL", is a popular strategy
video-game that is available to Windows/MacOS users. LoL consists 2 teams facing
each other, where the main objective is to destroy the opposing teams home base
while facing obstacles throughout the way. Perks & gold are able to be obtained
by players/teams through completing tasks such as eliminating players, enemy
minions, or dragons. Players then spend the gold to purchase items that help
deal more damage towards enemies.

League of Legends offers different game modes, such as ranked. A ranked game on
average lasts 30-45 minutes. In this game mode, players are given a rank based
off of the number of wins + the number of games played. "Diamond" is one of the
highest ranks a player may obtain and is known to be extremely competitive.

// Once you begin modeling, how well would your prediction algorithm
// need to perform to be considered useful to these third parties?

This dataset contains 19 features per team (blue & red), 38 in total.
Features include the total number of kills, CS (creep score), dragons, etc.
The winner of the match is also one of the features in the data set that is
listed as "blueWins". As for the instances, there is a total of 9,980 diamond
ranked games.

-------------------------------------
Dataset : https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min

Question Of Interest : Which features are tied to winning games?

### 2. Data Understanding
### 3. Data Visualization
#### 3.1 Data Exploration
#### 3.2 Data Relationship Exploration
### 4. Dimensionality Reduction
#### Reference